**1. Pemrosesan Bahasa Alami (NLP) dan RNN**

* **Turing Test**: Alan Turing pada tahun 1950 mengusulkan Turing Test, sebuah tugas linguistik untuk mengevaluasi kemampuan mesin menyamai kecerdasan manusia. Mesin, atau chatbot, dirancang untuk menipu lawan bicaranya agar mengira ia adalah manusia. Meskipun ada kelemahan dalam tes ini (misalnya, aturan yang dikodekan secara keras dapat menipu manusia yang tidak curiga), tes ini menyoroti bahwa penguasaan bahasa adalah kemampuan kognitif terbesar *Homo sapiens*.
* **Pendekatan Umum**: Pendekatan umum untuk tugas-tugas bahasa alami adalah menggunakan *Recurrent Neural Networks* (RNNs).
* **Jenis-jenis RNN dalam Konteks NLP**:
    * **Character RNN**: Melatih RNN untuk memprediksi karakter berikutnya dalam sebuah kalimat. Ini memungkinkan pembuatan teks asli dan menunjukkan cara membangun TensorFlow Dataset pada urutan yang sangat panjang.
        * **Stateless RNN**: Belajar dari bagian teks acak pada setiap iterasi, tanpa informasi mengenai sisa teks.
        * **Stateful RNN**: Mempertahankan *hidden state* antar iterasi pelatihan dan melanjutkan membaca dari tempat terakhir, memungkinkannya mempelajari pola yang lebih panjang.
    * **Word-level RNN**: Mengobati kalimat sebagai urutan kata, bukan karakter, untuk tugas-tugas seperti analisis sentimen.

**2. Membangun Dataset untuk Character RNN**

* **Mengunduh dan Mengkodekan Teks**: Teks (misalnya, karya Shakespeare) diunduh menggunakan `keras.utils.get_file()`. Setiap karakter kemudian dikodekan sebagai bilangan bulat. `Keras.preprocessing.text.Tokenizer` digunakan untuk menemukan semua karakter unik dan memetakannya ke ID karakter (mulai dari 1). `char_level=True` diatur untuk pengkodean tingkat karakter.
* **Membagi Dataset Sekuensial**: Penting untuk menghindari tumpang tindih antara *training set*, *validation set*, dan *test set*. Untuk data sekuensial, biasanya dibagi berdasarkan waktu (misalnya, tahun 2000-2012 untuk pelatihan, 2013-2015 untuk validasi, 2016-2018 untuk pengujian). Namun, dalam beberapa kasus, pembagian dapat dilakukan berdasarkan dimensi lain, seperti perusahaan dalam data keuangan, meskipun korelasi antar entitas dapat bias pengukuran *generalization error*. Pembagian berdasarkan waktu seringkali lebih aman, dengan asumsi *time series* bersifat stasioner.
* **Memecah Dataset Sekuensial menjadi Banyak *Window***: Karena *training set* dapat terdiri dari urutan jutaan karakter, RNN tidak dapat dilatih langsung pada seluruhnya. Metode `dataset.window()` digunakan untuk mengubah urutan panjang ini menjadi banyak *window* teks yang lebih kecil. Ini disebut *truncated backpropagation through time*.
    * `shift=1` digunakan untuk menghasilkan *window* yang tumpang tindih, memaksimalkan *training set*.
    * `drop_remainder=True` memastikan semua *window* memiliki panjang yang sama.
    * `flat_map()` digunakan untuk mengubah *nested dataset* (dataset yang berisi dataset lain) menjadi *flat dataset* yang berisi *tensor*.
    * *Window* kemudian diacak (`shuffle()`) dan di-batch (`batch()`), dan masukan (`inputs`) dipisahkan dari target (`targets`).
    * Setiap karakter dienkode menggunakan *one-hot vector* karena jumlah karakter yang berbeda relatif sedikit (39).
    * Terakhir, `prefetch(1)` ditambahkan untuk *prefetching*.

**3. Membangun dan Melatih Model Char-RNN**

* **Arsitektur Model**: RNN dengan 2 layer GRU (masing-masing 128 unit, 20% *dropout* pada masukan dan *hidden state*) dan layer *TimeDistributed Dense* dengan 39 unit (jumlah karakter unik) dan fungsi aktivasi *softmax*.
* **Kompilasi dan Pelatihan**: Model dikompilasi dengan *loss* "sparse_categorical_crossentropy" dan optimizer "adam".

**4. Menggunakan Model Char-RNN**

* **Preprocessing Input**: Teks masukan harus di-preprocess menggunakan fungsi `preprocess` yang mengodekan teks menjadi ID karakter dan mengonversinya menjadi *one-hot vector*.
* **Memprediksi Karakter Berikutnya**: Model dapat memprediksi karakter berikutnya.
* **Menghasilkan Teks**: Untuk menghasilkan teks baru, karakter berikutnya dipilih secara acak berdasarkan probabilitas yang diestimasi menggunakan `tf.random.categorical()`. *Temperature* digunakan untuk mengontrol keragaman teks yang dihasilkan: *temperature* mendekati 0 akan mendukung karakter berprobabilitas tinggi, sementara *temperature* sangat tinggi akan memberikan semua karakter probabilitas yang sama. Fungsi `next_char()` dan `complete_text()` digunakan untuk ini.
* **Peningkatan Kinerja**: Untuk teks yang lebih meyakinkan, dapat mencoba lebih banyak layer GRU dan neuron per layer, pelatihan lebih lama, dan penambahan regularisasi (misalnya, `recurrent_dropout=0.3`).

**5. Stateful RNN**

* **Perbedaan dari Stateless RNN**: *Stateful RNN* mempertahankan *final state* setelah memproses satu *training batch* dan menggunakannya sebagai *initial state* untuk *training batch* berikutnya. Ini memungkinkan model mempelajari pola jangka panjang meskipun hanya melakukan *backpropagation* melalui urutan pendek.
* **Membangun Stateful RNN**:
    * Menggunakan urutan masukan sekuensial dan tidak tumpang tindih (`shift=n_steps` dan tanpa `shuffle()`).
    * Pengaturan `stateful=True` pada setiap *recurrent layer* dan `batch_input_shape` pada layer pertama diperlukan.
    * *Callback* `ResetStatesCallback` digunakan untuk mengatur ulang status di awal setiap *epoch*.
* **Pembatasan**: Setelah dilatih, model *stateful* hanya dapat digunakan untuk prediksi dengan ukuran *batch* yang sama seperti saat pelatihan. Untuk menghindari batasan ini, salin bobot model *stateful* ke model *stateless* yang identik.

**6. Analisis Sentimen**

* **IMDb Reviews Dataset**: Dataset ini adalah "hello world" NLP, terdiri dari 50.000 ulasan film berbahasa Inggris (25.000 untuk pelatihan, 25.000 untuk pengujian) dengan target biner (negatif/positif).
* **Preprocessing Data Teks**:
    * Dataset IMDb sudah di-preprocess (kata-kata diwakili oleh ID bilangan bulat, tanda baca dihapus, dikonversi ke huruf kecil, diindeks berdasarkan frekuensi).
    * ID 0, 1, dan 2 adalah spesial: token *padding*, *start-of-sequence* (SSS), dan kata tidak dikenal (*unknown words*).
    * Untuk proyek nyata, `Tokenizer` dengan `char_level=False` (default) dapat digunakan.
    * **Subword Tokenization**: Untuk mengatasi batasan tokenisasi berbasis spasi (terutama untuk bahasa non-Inggris atau kata-kata yang tidak biasa), teknik *subword tokenization* seperti yang diperkenalkan oleh Taku Kudo (SentencePiece) atau Rico Sennrich et al. (Byte Pair Encoding) dapat digunakan. TensorFlow menyediakan library TF.Text yang mengimplementasikan berbagai strategi tokenisasi.
    * **Preprocessing dengan TensorFlow Operations**: Untuk menyertakan preprocessing dalam model itu sendiri (untuk deployment), fungsi preprocessing dapat ditulis menggunakan operasi TensorFlow seperti `tf.strings.substr`, `tf.strings.regex_replace`, dan `tf.strings.split`.
    * **Membangun Kosakata**: `collections.Counter` digunakan untuk menghitung frekuensi kata, dan kosakata dipotong menjadi 10.000 kata yang paling umum.
    * **Memetakan Kata ke ID**: `tf.lookup.StaticVocabularyTable` digunakan untuk memetakan kata ke ID-nya (indeks dalam kosakata), dengan *out-of-vocabulary* (oov) *buckets* untuk kata-kata yang tidak ditemukan.
* **Arsitektur Model Sentimen Analisis**:
    * **Embedding Layer**: Mengkonversi ID kata menjadi *embeddings*. Matriks *embedding* memiliki satu baris per ID kata dan satu kolom per dimensi *embedding* (misalnya, 128 dimensi).
    * Dua layer GRU, dengan GRU kedua hanya mengembalikan keluaran dari langkah waktu terakhir.
    * Layer keluaran adalah neuron tunggal dengan fungsi aktivasi *sigmoid* untuk memprediksi probabilitas sentimen positif.
    * Kompilasi menggunakan *loss* "binary_crossentropy" dan optimizer "adam".

**7. Masking**

* **Tujuan**: Memberi tahu model untuk mengabaikan token *padding* (ID 0) sehingga dapat fokus pada data yang relevan.
* **Cara Kerja**: Menambahkan `mask_zero=True` saat membuat layer *Embedding*. Layer *Embedding* membuat *mask tensor* yang secara otomatis disebarkan ke semua layer berikutnya selama dimensi waktu dipertahankan. Layer berulang akan mengabaikan langkah waktu yang di-*mask* (misalnya, menyalin keluaran dari langkah waktu sebelumnya). Jika *mask* sampai ke keluaran, itu akan diterapkan pada *loss* juga, sehingga langkah waktu yang di-*mask* tidak berkontribusi pada *loss*.
* **Keterbatasan**: Implementasi yang dioptimalkan untuk GPU (berdasarkan Nvidia cuDNN) tidak mendukung *masking*, sehingga *fallback* ke implementasi yang lebih lambat.
* **Dukungan Masking**: Semua layer yang menerima *mask* harus mendukung *masking* (memiliki atribut `supports_masking=True`).
* **Masking Manual**: Untuk model yang lebih kompleks (misalnya, mencampur Conv1D dengan layer berulang), *masking* mungkin perlu dihitung secara eksplisit dan diteruskan ke layer yang sesuai menggunakan *Functional API* atau *Subclassing API*.

**8. Menggunakan Ulang *Pretrained Embeddings***

* **TensorFlow Hub**: Memudahkan penggunaan kembali komponen model yang sudah dilatih sebelumnya (disebut *modules*). Cukup telusuri repositori TF Hub, temukan yang dibutuhkan, dan salin contoh kode.
* **Contoh**: Menggunakan `nnlm-en-dim50` *sentence embedding module*. Layer `hub.KerasLayer` mengunduh modul. Modul ini adalah *sentence encoder* yang mengubah string menjadi vektor (50 dimensi dalam contoh ini). Secara internal, ia mem-parsing string, menyematkan setiap kata menggunakan matriks *embedding* yang sudah dilatih sebelumnya pada korpus besar (Google News 7B), lalu menghitung rata-rata semua *word embeddings* untuk mendapatkan *sentence embedding*.
* Secara *default*, `hub.KerasLayer` tidak dapat dilatih, tetapi dapat diatur `trainable=True` untuk *fine-tuning*.

**9. Jaringan Encoder-Decoder untuk Neural Machine Translation (NMT)**

* **Tujuan**: Menerjemahkan kalimat dari satu bahasa ke bahasa lain (misalnya, Inggris ke Prancis).
* **Arsitektur**:
    * **Encoder**: Menerima kalimat masukan (misalnya, bahasa Inggris). Kalimat masukan dibalik sebelum dimasukkan ke *encoder* (misalnya, "I drink milk" menjadi "milk drink I") untuk memastikan bagian awal kalimat masukan dimasukkan terakhir ke *encoder*, yang berguna karena itu biasanya hal pertama yang perlu diterjemahkan oleh *decoder*.
    * **Decoder**: Mengeluarkan terjemahan (misalnya, bahasa Prancis). Terjemahan bahasa Prancis juga digunakan sebagai masukan ke *decoder*, tetapi digeser satu langkah ke belakang (token *start-of-sequence* (SOS) untuk kata pertama). *Decoder* diharapkan mengakhiri kalimat dengan token *end-of-sequence* (EOS).
    * **Word Embedding**: Setiap kata diwakili oleh ID-nya, kemudian layer *embedding* mengubahnya menjadi *word embedding* yang kemudian dimasukkan ke *encoder* dan *decoder*.
    * **Output Decoder**: Pada setiap langkah, *decoder* mengeluarkan skor untuk setiap kata dalam kosakata keluaran, dan layer *softmax* mengubah skor ini menjadi probabilitas. Kata dengan probabilitas tertinggi dikeluarkan.
    * **Inference Time**: Pada waktu inferensi, *decoder* menerima kata yang dikeluarkan pada langkah sebelumnya sebagai masukan.
* **Detail Implementasi**:
    * **Panjang Urutan Variabel**: Masking dapat digunakan. Untuk kalimat dengan panjang yang sangat berbeda, kalimat dapat dikelompokkan ke dalam *bucket* dengan panjang yang serupa dan di-*padding*.
    * **Mengabaikan Output Setelah EOS**: Output setelah token EOS harus diabaikan dan tidak berkontribusi pada *loss*.
    * **Kosakata Output Besar**: Untuk kosakata output yang besar, teknik *sampled softmax* (`tf.nn.sampled_softmax_loss()`) dapat digunakan selama pelatihan untuk menghindari perhitungan *softmax* yang lambat.
* **TensorFlow Addons**: Menyediakan alat *sequence-to-sequence* untuk membangun *Encoder-Decoder*.
    * `return_state=True` pada layer LSTM *encoder* untuk mendapatkan *final hidden state* yang diteruskan ke *decoder*.
    * `tfa.seq2seq.sampler.TrainingSampler()` memberi tahu *decoder* pada setiap langkah apa yang harus pura-pura menjadi keluaran sebelumnya (selama pelatihan, *embedding* dari token target sebelumnya).

**10. Bidirectional RNNs**

* **Keterbatasan RNN Reguler**: Layer rekuren reguler hanya melihat masukan masa lalu dan sekarang, bersifat "kausal".
* **Keuntungan Bidirectional RNN**: Untuk banyak tugas NLP, seperti NMT, seringkali lebih baik untuk melihat kata-kata berikutnya sebelum mengkodekan kata tertentu. *Bidirectional recurrent layer* menjalankan dua layer rekuren pada masukan yang sama (satu membaca dari kiri ke kanan, yang lain dari kanan ke kiri) dan menggabungkan keluarannya (biasanya dengan menyambungkannya).
* **Implementasi di Keras**: Pembungkus `keras.layers.Bidirectional` digunakan untuk layer rekuren.

**11. Beam Search**

* **Masalah *Greedy Decoding***: Dengan mengeluarkan kata yang paling mungkin pada setiap langkah (*greedy decoding*), model dapat membuat kesalahan yang tidak dapat diperbaiki kemudian, menghasilkan terjemahan yang suboptimal.
* **Solusi Beam Search**: Melacak daftar pendek dari *k* kalimat yang paling menjanjikan (misalnya, tiga teratas) dan pada setiap langkah *decoder*, mencoba memperluasnya dengan satu kata, hanya menyimpan *k* kalimat yang paling mungkin. *k* adalah *beam width*.
* **Implementasi dengan TensorFlow Addons**: `tfa.seq2seq.beam_search_decoder.BeamSearchDecoder` digunakan, yang membungkus semua klon *decoder*.

**12. Attention Mechanisms**

* **Masalah Memori Jangka Pendek RNN**: Jalur dari kata masukan ke terjemahannya bisa sangat panjang, membebani memori jangka pendek RNN.
* **Tujuan Attention**: Memungkinkan *decoder* untuk fokus pada kata-kata yang sesuai (yang dienkode oleh *encoder*) pada setiap langkah waktu. Ini membuat jalur dari kata masukan ke terjemahannya jauh lebih pendek, mengurangi dampak keterbatasan memori jangka pendek RNN.
* **Revolusi NLP**: Mekanisme *attention* merevolusi NMT (dan NLP secara umum), memungkinkan peningkatan signifikan dalam *state of the art*, terutama untuk kalimat panjang.
* **Arsitektur**:
    * **Encoder Outputs**: Semua keluaran *encoder* dikirim ke *decoder*.
    * **Weighted Sum**: Pada setiap langkah waktu, *memory cell decoder* menghitung jumlah tertimbang dari semua keluaran *encoder*. Ini menentukan kata mana yang akan difokuskan pada langkah ini.
    * **Alignment Model (Attention Layer)**: Jaringan saraf kecil yang dilatih bersama dengan model *Encoder-Decoder* untuk menghasilkan bobot $\alpha_{(t,i)}$ (bobot keluaran *encoder* ke-i pada langkah waktu *decoder* ke-t).
        * Menerima masukan dari semua keluaran *encoder* yang disambung dengan *hidden state* *decoder* sebelumnya (misalnya, $h_{(2)}$).
        * Mengeluarkan skor (atau energi) untuk setiap keluaran *encoder*, yang mengukur seberapa baik setiap keluaran selaras dengan *hidden state* *decoder* sebelumnya.
        * Fungsi *softmax* diterapkan pada semua skor untuk mendapatkan bobot akhir untuk setiap keluaran *encoder*.
* **Jenis-jenis Mekanisme Attention**:
    * **Bahdanau Attention (Concatenative/Additive Attention)**: Disebut demikian karena menggabungkan keluaran *encoder* dengan *hidden state* *decoder* sebelumnya.
    * **Luong Attention (Multiplicative Attention)**: Menghitung *dot product* antara keluaran *encoder* dan *hidden state* *decoder* (saat ini, $h_{(t)}$, bukan $h_{(t-1)}$) sebagai ukuran kesamaan. Umumnya berkinerja lebih baik daripada *concatenative attention*.
        * `tfa.seq2seq.attention_wrapper.LuongAttention` digunakan untuk implementasi.
* **Computational Complexity**: Model *attention* akan membutuhkan perhitungan sekitar $n^2$ bobot jika kalimat masukan panjangnya *n* kata.
* **Visual Attention**: Aplikasi *attention* di luar NMT, seperti menghasilkan *caption* gambar. Model fokus pada bagian gambar yang relevan saat menghasilkan setiap kata dalam *caption*.
* **Explainability**: *Attention mechanisms* membuat model lebih mudah dipahami mengapa ia menghasilkan keluaran tertentu, berguna untuk *debugging* dan persyaratan hukum.

**13. Transformer Architecture: "Attention Is All You Need"**

* **Inovasi**: Diperkenalkan pada tahun 2017 oleh Google, Transformer secara signifikan meningkatkan *state of the art* di NMT tanpa menggunakan layer rekuren atau konvolusional, hanya mekanisme *attention*.
* **Keuntungan**: Lebih cepat dilatih dan lebih mudah di-parallellkan.
* **Arsitektur (Komponen Utama)**:
    * **Encoder (Bagian Kiri)**: Menerima *batch* kalimat masukan (ID kata) dan mengkodekan setiap kata menjadi representasi 512 dimensi. Terdiri dari *N* layer yang ditumpuk.
    * **Decoder (Bagian Kanan)**: Selama pelatihan, menerima kalimat target (ID kata) yang digeser satu langkah ke kanan (dengan token SOS di awal). Juga menerima keluaran dari *encoder*. Terdiri dari *N* layer yang ditumpuk. Mengeluarkan probabilitas untuk setiap kata berikutnya.
    * **Inference Time**: *Decoder* dipanggil berulang kali, memprediksi satu kata lagi di setiap putaran, hingga token EOS dikeluarkan.
    * **Komponen Familiar**: Dua layer *embedding*, $5 \times N$ *skip connections* (diikuti oleh *layer normalization*), $2 \times N$ blok "Feed Forward" (dua layer *dense*), dan layer keluaran *dense* dengan *softmax*. Semua layer ini bersifat *time-distributed*.
* **Komponen Baru**:
    * **Multi-Head Attention (Self-Attention)**:
        * **Encoder**: Mengkodekan hubungan setiap kata dengan setiap kata lain dalam kalimat yang sama, memberi perhatian lebih pada yang paling relevan.
        * **Decoder (Masked Multi-Head Attention)**: Sama seperti *self-attention* pada *encoder*, tetapi setiap kata hanya diizinkan untuk memperhatikan kata-kata yang terletak sebelumnya.
        * **Decoder (Upper Multi-Head Attention)**: *Decoder* memberi perhatian pada kata-kata dalam kalimat masukan.
    * **Positional Embeddings**: Vektor padat yang mewakili posisi kata dalam kalimat. Ditambahkan ke *word embeddings* untuk memberikan model informasi posisi kata, karena layer *Multi-Head Attention* tidak mempertimbangkan urutan atau posisi kata.
        * Dapat dipelajari atau menggunakan *fixed positional embeddings* yang didefinisikan dengan fungsi sinus dan kosinus dari frekuensi yang berbeda.
        * Solusi sinus/kosines dapat diperluas ke kalimat yang sangat panjang dan memberikan model akses ke posisi absolut dan relatif kata.
        * Implementasi kelas `PositionalEncoding` di TensorFlow.
* **Scaled Dot-Product Attention (Dasar Multi-Head Attention)**:
    * Konsep *differentiable dictionary lookup*: model mencari informasi yang relevan dari masukan berdasarkan *query*.
    * Kesamaan diukur menggunakan *dot product* antara *query* (Q) dan *key* (K), diskalakan oleh $\sqrt{d_{keys}}$, lalu melewati *softmax* untuk mendapatkan bobot. Bobot ini digunakan untuk menjumlahkan *value* (V) secara tertimbang.
    * Masking dapat digunakan untuk mengabaikan pasangan *key/value* tertentu.
    * Dalam *encoder*, Q, K, dan V sama dengan daftar kata dalam kalimat masukan (*self-attention*). Dalam layer *attention* yang di-*mask* pada *decoder*, Q, K, dan V sama dengan daftar kata dalam kalimat target, dengan *mask* untuk mencegah melihat kata-kata di masa depan. Pada layer *attention* atas *decoder*, K dan V adalah *encodings* kata dari *encoder*, dan Q adalah *encodings* kata dari *decoder*.
    * `keras.layers.Attention` mengimplementasikan *Scaled Dot-Product Attention*.
* **Arsitektur Multi-Head Attention**:
    * Terdiri dari beberapa layer *Scaled Dot-Product Attention*, masing-masing didahului oleh transformasi linier dari *value*, *key*, dan *query*.
    * Semua keluaran disambung dan melewati transformasi linier akhir.
    * **Intuisi**: Memungkinkan model untuk menerapkan banyak proyeksi berbeda dari representasi kata ke *subspace* yang berbeda, masing-masing fokus pada subset karakteristik kata (misalnya, satu *head* fokus pada apakah kata itu kata kerja, yang lain pada tensesnya).

**14. Inovasi Terbaru dalam Model Bahasa (2018-2019)**

* **"ImageNet moment for NLP" (2018)**: Kemajuan luar biasa dengan arsitektur berbasis LSTM dan Transformer yang semakin besar dilatih pada dataset yang sangat besar.
* **ELMo (Embeddings from Language Models)**: Memperkenalkan *contextualized word embeddings* yang dipelajari dari *internal states* model bahasa *bidirectional* yang dalam. "Queen" akan memiliki *embedding* yang berbeda dalam "Queen of the United Kingdom" dan "queen bee".
* **ULMFiT (Universal Language Model Fine-Tuning for Text Classification)**: Mendemonstrasikan efektivitas *unsupervised pretraining* untuk tugas NLP. Model bahasa LSTM dilatih menggunakan *self-supervised learning* pada korpus teks besar, kemudian di-*fine-tune* untuk berbagai tugas. Model ini mengungguli *state of the art* pada tugas klasifikasi teks. Bahkan dengan hanya 100 contoh berlabel, dapat mencapai kinerja yang sama dengan model yang dilatih dari awal dengan 10.000 contoh.
* **GPT (Generative Pre-Training)**: Mendemonstrasikan efektivitas *unsupervised pretraining* menggunakan arsitektur seperti Transformer (tumpukan 12 modul Transformer dengan hanya layer *Masked Multi-Head Attention*). Di-*fine-tune* pada berbagai tugas bahasa (klasifikasi teks, *entailment*, kesamaan, *question answering*) dengan sedikit adaptasi.
    * **GPT-2 (2019)**: Arsitektur yang sangat mirip tetapi lebih besar (lebih dari 1,5 miliar parameter). Menunjukkan kinerja yang baik pada banyak tugas tanpa *fine-tuning* (*zero-shot learning*).
* **BERT (Bidirectional Encoder Representations from Transformers)**: Mendemonstrasikan efektivitas *self-supervised pretraining* pada korpus besar, menggunakan arsitektur mirip GPT tetapi layer *Non-Masked Multi-Head Attention* (seperti pada *encoder* Transformer). Ini membuatnya *bidirectional*.
    * **Dua Tugas Pretraining Utama**:
        * **Masked Language Model (MLM)**: Kata-kata dalam kalimat di-*mask* (15% probabilitas), dan model dilatih untuk memprediksi kata-kata yang di-*mask*.
        * **Next Sentence Prediction (NSP)**: Model dilatih untuk memprediksi apakah dua kalimat berurutan atau tidak. Ini meningkatkan kinerja model saat di-*fine-tune* pada tugas seperti *question answering* atau *entailment*.
* **Tren Utama**: Tokenisasi *subword* yang lebih baik, pergeseran dari LSTM ke Transformer, dan *pretraining* model bahasa universal menggunakan *self-supervised learning*, lalu *fine-tuning* dengan sedikit atau tanpa perubahan arsitektur.